In [1]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import gc

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

In [111]:
from General import *
from ReadingTheDataUtils import *
from Classifiers import *
from PreproccesUtils import *
from PerformanceEvalutionUtils import *

# Make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from pathlib import Path

ROOT_PATH = Path('..')
SRC_PATH = Path('.')
DATA_PATH = ROOT_PATH / 'data'
CSV_PATH = DATA_PATH / 'ExtraSensory.per_uuid_features_labels'
CSV_SUFFIX = '.features_labels.csv'
ORIGINAL_LABLES_CSV_PATH = DATA_PATH / 'ExtraSensory.per_uuid_original_labels'
ORIGINAL_LABLES_CSV_SUFFIX = '.original_labels.csv'
FOLD_PATH = DATA_PATH / 'cv_5_folds'

# Data Exploration

In [4]:
data = pd.read_csv(DATA_PATH / 'dataset.csv', index_col='uuid')

In [5]:
data.head()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label,label_name
uuid,,,,,,,,,,,,,,,,,,,,,
00EABED2-271D-49D8-B599-1D4A09240601,1444079161,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,6.684605,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING
00EABED2-271D-49D8-B599-1D4A09240601,1444079221,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,6.684603,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING
00EABED2-271D-49D8-B599-1D4A09240601,1444079281,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,6.684605,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING
00EABED2-271D-49D8-B599-1D4A09240601,1444079341,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,6.684605,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING
00EABED2-271D-49D8-B599-1D4A09240601,1444079431,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,6.683904,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING


In [6]:
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 377346 entries, 00EABED2-271D-49D8-B599-1D4A09240601 to FDAA70A1-42A3-4E3F-9AE3-3FDA412E03BF
Data columns (total 228 columns):
timestamp                                                                  377346 non-null int64
raw_acc:magnitude_stats:mean                                               377056 non-null float64
raw_acc:magnitude_stats:std                                                377056 non-null float64
raw_acc:magnitude_stats:moment3                                            377056 non-null float64
raw_acc:magnitude_stats:moment4                                            377056 non-null float64
raw_acc:magnitude_stats:percentile25                                       377056 non-null float64
raw_acc:magnitude_stats:percentile50                                       377056 non-null float64
raw_acc:magnitude_stats:percentile75                                       377056 non-null float64
raw_acc:magnitude_stats:value_entropy 

In [7]:
data['label'] = data['label'].astype('category')

for col in data.columns:
    if col.startswith('dicrete'):
        data[col] = data[col].astype('category')  

# Model 

### Learn all classifiers by folds 

In [8]:
train_folds_list, test_folds_list = get_folds_list(FOLD_PATH)

In [9]:
%%time
classifiers = dict()

for train_fold_lst, test_fold_lst in zip(train_folds_list, test_folds_list):
    gc.collect()
    
    # Preprocess the data
    train_fold_df, test_fold_df = get_folds_train_and_test(data, train_fold_lst, test_fold_lst)
    X_fold_train, X_fold_test, y_fold_train, y_fold_test = \
                split_fold_data_to_features_and_labels(train_fold_df, test_fold_df)
    standard_X_train, standard_X_test = standard_data_scaling(X_fold_train, X_fold_test)
        
    handle_nulls_in_X(standard_X_train, standard_X_test)
    
    # Learn all models sync
#     models_tuple =learn_all_models_sync(standard_X_train, y_fold_train)

    # Learn all model async
    models_tuple = learn_all_models_async(standard_X_train, y_fold_train, i_c_score_grid_search=True)
    
    # Put each fold result in the classifiers dict
    classifiers.setdefault('single_sensor_classifier', []).append(models_tuple[0])
    classifiers.setdefault('early_fusion_classifier', []).append(models_tuple[1])

2019-10-14 17:45:41,026 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - start single sensor model
2019-10-14 17:45:41,031 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - inside the main loop
2019-10-14 17:45:41,032 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - Acc
2019-10-14 17:45:41,570 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - starting a grid search
2019-10-14 17:48:29,213 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - finished the grid search
2019-10-14 17:49:17,661 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - inside the main loop
2019-10-14 17:49:17,662 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - Gyro
2019-10-14 17:49:18,138 - DEBUG - get_single_sensor_classifier - Classifier

2019-10-14 18:25:38,817 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - inside the main loop
2019-10-14 18:25:38,818 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - WAcc
2019-10-14 18:25:39,474 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - starting a grid search
2019-10-14 18:28:28,848 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - finished the grid search
2019-10-14 18:28:55,584 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - inside the main loop
2019-10-14 18:28:55,585 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - Loc
2019-10-14 18:28:55,986 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - starting a grid search
2019-10-14 18:30:06,743 - DEBUG - get_single_sensor_classifier 

2019-10-14 19:00:25,451 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - Aud
2019-10-14 19:00:25,915 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - starting a grid search
2019-10-14 19:01:38,478 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - finished the grid search
2019-10-14 19:01:53,498 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - inside the main loop
2019-10-14 19:01:53,499 - DEBUG - get_single_sensor_classifier - Classifiers - get_single_sensor_classifier - PS
2019-10-14 19:01:54,022 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - starting a grid search
2019-10-14 19:04:03,912 - DEBUG - get_single_sensor_classifier - Classifiers - single_label_logistic_regression_classifier - finished the grid search
2019-10-14 19:04:20,245 - DEBUG - get_single_s

In [10]:
classifiers

2019-10-14 19:04:20,771 - DEBUG - MainThread - Classifiers - <module> - First time initialize logger!


{'single_sensor_classifier': [{'Acc': LogisticRegression(C=1, class_weight='balanced', dual=False,
             fit_intercept=True, intercept_scaling=1, max_iter=1000,
             multi_class='warn', n_jobs=-1, penalty='l2', random_state=None,
             solver='lbfgs', tol=0.0001, verbose=0, warm_start=False),
   'Gyro': LogisticRegression(C=0.001, class_weight='balanced', dual=False,
             fit_intercept=True, intercept_scaling=1, max_iter=1000,
             multi_class='warn', n_jobs=-1, penalty='l2', random_state=None,
             solver='lbfgs', tol=0.0001, verbose=0, warm_start=False),
   'WAcc': LogisticRegression(C=0.001, class_weight='balanced', dual=False,
             fit_intercept=True, intercept_scaling=1, max_iter=1000,
             multi_class='warn', n_jobs=-1, penalty='l2', random_state=None,
             solver='lbfgs', tol=0.0001, verbose=0, warm_start=False),
   'Loc': LogisticRegression(C=100, class_weight='balanced', dual=False,
             fit_intercep

# Performance evaluation

<tr>
<td>
<dl><dt>condition positive (P)</dt>
<dd>the number of real positive cases in the data</dd>
<dt>condition negative (N)</dt>
<dd>the number of real negative cases in the data</dd></dl>
<hr>
<dl><dt>true positive (TP)</dt>
<dd>eqv. with hit</dd>
<dt>true negative (TN)</dt>
<dd>eqv. with correct rejection</dd>
<dt>false positive (FP)</dt>
<dd>eqv. with <a href="/wiki/False_alarm" title="False alarm">false alarm</a>, <a href="/wiki/Type_I_error" class="mw-redirect" title="Type I error">Type I error</a></dd>
<dt>false negative (FN)</dt>
<dd>eqv. with miss, <a href="/wiki/Type_II_error" class="mw-redirect" title="Type II error">Type II error</a></dd></dl>
<hr>
<dl><dt><a href="/wiki/Sensitivity_(test)" class="mw-redirect" title="Sensitivity (test)">sensitivity</a>, <a href="/wiki/Precision_and_recall#Recall" title="Precision and recall">recall</a>, <a href="/wiki/Hit_rate" title="Hit rate">hit rate</a>, or <a href="/wiki/Sensitivity_(test)" class="mw-redirect" title="Sensitivity (test)">true positive rate</a> (TPR)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {TPR} ={\frac {\mathrm {TP} }{\mathrm {P} }}={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} }}=1-\mathrm {FNR} }">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">T</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">T</mi>
              <mi mathvariant="normal">P</mi>
            </mrow>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">P</mi>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">T</mi>
              <mi mathvariant="normal">P</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mn>1</mn>
        <mo>−<!-- − --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">F</mi>
          <mi mathvariant="normal">N</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {TPR} ={\frac {\mathrm {TP} }{\mathrm {P} }}={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} }}=1-\mathrm {FNR} }</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/f02ea353bf60bfdd9557d2c98fe18c34cd8db835" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:37.537ex; height:5.343ex;" alt="{\displaystyle \mathrm {TPR} ={\frac {\mathrm {TP} }{\mathrm {P} }}={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} }}=1-\mathrm {FNR} }"></span></dd>
<dt><a href="/wiki/Specificity_(tests)" class="mw-redirect" title="Specificity (tests)">specificity</a>, <a href="/wiki/Specificity_(tests)" class="mw-redirect" title="Specificity (tests)">selectivity</a> or <a href="/wiki/Specificity_(tests)" class="mw-redirect" title="Specificity (tests)">true negative rate</a> (TNR)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {TNR} ={\frac {\mathrm {TN} }{\mathrm {N} }}={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FP} }}=1-\mathrm {FPR} }">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">T</mi>
          <mi mathvariant="normal">N</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">T</mi>
              <mi mathvariant="normal">N</mi>
            </mrow>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">N</mi>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">T</mi>
              <mi mathvariant="normal">N</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mn>1</mn>
        <mo>−<!-- − --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">F</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {TNR} ={\frac {\mathrm {TN} }{\mathrm {N} }}={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FP} }}=1-\mathrm {FPR} }</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/8f2c867f0641e498ec8a59de63697a3a45d66b07" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:37.697ex; height:5.343ex;" alt="{\displaystyle \mathrm {TNR} ={\frac {\mathrm {TN} }{\mathrm {N} }}={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FP} }}=1-\mathrm {FPR} }"></span></dd>
<dt><a href="/wiki/Information_retrieval#Precision" title="Information retrieval">precision</a> or <a href="/wiki/Positive_predictive_value" class="mw-redirect" title="Positive predictive value">positive predictive value</a> (PPV)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {PPV} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FP} }}=1-\mathrm {FDR} }">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">V</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">T</mi>
              <mi mathvariant="normal">P</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mn>1</mn>
        <mo>−<!-- − --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">F</mi>
          <mi mathvariant="normal">D</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {PPV} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FP} }}=1-\mathrm {FDR} }</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/d854b1544fc77735d575ce0d30e34d7f1eacf707" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:30.151ex; height:5.343ex;" alt="{\displaystyle \mathrm {PPV} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FP} }}=1-\mathrm {FDR} }"></span></dd>
<dt><a href="/wiki/Negative_predictive_value" class="mw-redirect" title="Negative predictive value">negative predictive value</a> (NPV)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {NPV} ={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FN} }}=1-\mathrm {FOR} }">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">N</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">V</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">T</mi>
              <mi mathvariant="normal">N</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mn>1</mn>
        <mo>−<!-- − --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">F</mi>
          <mi mathvariant="normal">O</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {NPV} ={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FN} }}=1-\mathrm {FOR} }</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/2cbe32c31443d1be4b43f2a8242f4a980b9fbf67" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:30.664ex; height:5.343ex;" alt="{\displaystyle \mathrm {NPV} ={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FN} }}=1-\mathrm {FOR} }"></span></dd>
<dt>miss rate or <a href="/wiki/Type_I_and_type_II_errors#False_positive_and_false_negative_rates" title="Type I and type II errors">false negative rate</a> (FNR)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {FNR} ={\frac {\mathrm {FN} }{\mathrm {P} }}={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TP} }}=1-\mathrm {TPR} }">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">F</mi>
          <mi mathvariant="normal">N</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">F</mi>
              <mi mathvariant="normal">N</mi>
            </mrow>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">P</mi>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">F</mi>
              <mi mathvariant="normal">N</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mn>1</mn>
        <mo>−<!-- − --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">T</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {FNR} ={\frac {\mathrm {FN} }{\mathrm {P} }}={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TP} }}=1-\mathrm {TPR} }</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/2af486535eb235ed28c3063ed05fd21657b28410" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:37.537ex; height:5.343ex;" alt="{\displaystyle \mathrm {FNR} ={\frac {\mathrm {FN} }{\mathrm {P} }}={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TP} }}=1-\mathrm {TPR} }"></span></dd>
<dt><a href="/wiki/Information_retrieval#Fall-out" title="Information retrieval">fall-out</a> or <a href="/wiki/Information_retrieval#Fall-out" title="Information retrieval">false positive rate</a> (FPR)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {FPR} ={\frac {\mathrm {FP} }{\mathrm {N} }}={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TN} }}=1-\mathrm {TNR} }">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">F</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">F</mi>
              <mi mathvariant="normal">P</mi>
            </mrow>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">N</mi>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">F</mi>
              <mi mathvariant="normal">P</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mn>1</mn>
        <mo>−<!-- − --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">T</mi>
          <mi mathvariant="normal">N</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {FPR} ={\frac {\mathrm {FP} }{\mathrm {N} }}={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TN} }}=1-\mathrm {TNR} }</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/422d06161964ca90602ec8712cd211cb0d80da19" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:37.376ex; height:5.343ex;" alt="{\displaystyle \mathrm {FPR} ={\frac {\mathrm {FP} }{\mathrm {N} }}={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TN} }}=1-\mathrm {TNR} }"></span></dd>
<dt><a href="/wiki/False_discovery_rate" title="False discovery rate">false discovery rate</a> (FDR)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {FDR} ={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TP} }}=1-\mathrm {PPV} }">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">F</mi>
          <mi mathvariant="normal">D</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">F</mi>
              <mi mathvariant="normal">P</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mn>1</mn>
        <mo>−<!-- − --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">V</mi>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {FDR} ={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TP} }}=1-\mathrm {PPV} }</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/bbf60523d63befc2537b39d4a0316d3945efe341" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:30.151ex; height:5.343ex;" alt="{\displaystyle \mathrm {FDR} ={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TP} }}=1-\mathrm {PPV} }"></span></dd>
<dt><a href="/wiki/Positive_and_negative_predictive_values" title="Positive and negative predictive values">false omission rate</a> (FOR)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {FOR} ={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TN} }}=1-\mathrm {NPV} }">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">F</mi>
          <mi mathvariant="normal">O</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">F</mi>
              <mi mathvariant="normal">N</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mn>1</mn>
        <mo>−<!-- − --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">N</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">V</mi>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {FOR} ={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TN} }}=1-\mathrm {NPV} }</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/13c18dbde910630a908207ce7c72a94cf67ce942" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:30.664ex; height:5.343ex;" alt="{\displaystyle \mathrm {FOR} ={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TN} }}=1-\mathrm {NPV} }"></span></dd>
<dt>Threat score (TS) or Critical Success Index (CSI)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {TS} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} +\mathrm {FP} }}}">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">T</mi>
          <mi mathvariant="normal">S</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow class="MJX-TeXAtom-ORD">
              <mi mathvariant="normal">T</mi>
              <mi mathvariant="normal">P</mi>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {TS} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} +\mathrm {FP} }}}</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/c1c63ac8f01a87f0a32dd640f82fe6772a32003b" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:22.208ex; height:5.343ex;" alt="{\displaystyle \mathrm {TS} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} +\mathrm {FP} }}}"></span></dd></dl>
<hr>
<dl><dt><a href="/wiki/Accuracy" class="mw-redirect" title="Accuracy">accuracy</a> (ACC)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {ACC} ={\frac {\mathrm {TP} +\mathrm {TN} }{\mathrm {P} +\mathrm {N} }}={\frac {\mathrm {TP} +\mathrm {TN} }{\mathrm {TP} +\mathrm {TN} +\mathrm {FP} +\mathrm {FN} }}}">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">A</mi>
          <mi mathvariant="normal">C</mi>
          <mi mathvariant="normal">C</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {ACC} ={\frac {\mathrm {TP} +\mathrm {TN} }{\mathrm {P} +\mathrm {N} }}={\frac {\mathrm {TP} +\mathrm {TN} }{\mathrm {TP} +\mathrm {TN} +\mathrm {FP} +\mathrm {FN} }}}</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/47deb47eb7ac214423d0a6afd05ec0af362fef9b" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:44.056ex; height:5.343ex;" alt="{\displaystyle \mathrm {ACC} ={\frac {\mathrm {TP} +\mathrm {TN} }{\mathrm {P} +\mathrm {N} }}={\frac {\mathrm {TP} +\mathrm {TN} }{\mathrm {TP} +\mathrm {TN} +\mathrm {FP} +\mathrm {FN} }}}"></span></dd>
<dt><a href="/wiki/F1_score" title="F1 score">F1 score</a></dt>
<dd>is the <a href="/wiki/Harmonic_mean#Harmonic_mean_of_two_numbers" title="Harmonic mean">harmonic mean</a> of <a href="/wiki/Information_retrieval#Precision" title="Information retrieval">precision</a> and <a href="/wiki/Sensitivity_(test)" class="mw-redirect" title="Sensitivity (test)">sensitivity</a></dd>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {F} _{1}=2\cdot {\frac {\mathrm {PPV} \cdot \mathrm {TPR} }{\mathrm {PPV} +\mathrm {TPR} }}={\frac {2\mathrm {TP} }{2\mathrm {TP} +\mathrm {FP} +\mathrm {FN} }}}">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <msub>
          <mrow class="MJX-TeXAtom-ORD">
            <mi mathvariant="normal">F</mi>
          </mrow>
          <mrow class="MJX-TeXAtom-ORD">
            <mn>1</mn>
          </mrow>
        </msub>
        <mo>=</mo>
        <mn>2</mn>
        <mo>⋅<!-- ⋅ --></mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">P</mi>
                <mi mathvariant="normal">P</mi>
                <mi mathvariant="normal">V</mi>
              </mrow>
              <mo>⋅<!-- ⋅ --></mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
                <mi mathvariant="normal">R</mi>
              </mrow>
            </mrow>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">P</mi>
                <mi mathvariant="normal">P</mi>
                <mi mathvariant="normal">V</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
                <mi mathvariant="normal">R</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow>
              <mn>2</mn>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
            </mrow>
            <mrow>
              <mn>2</mn>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
          </mfrac>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {F} _{1}=2\cdot {\frac {\mathrm {PPV} \cdot \mathrm {TPR} }{\mathrm {PPV} +\mathrm {TPR} }}={\frac {2\mathrm {TP} }{2\mathrm {TP} +\mathrm {FP} +\mathrm {FN} }}}</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/0e5f071c6418f444fadc9f5f9b0358beed3e094c" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.005ex; width:42.469ex; height:5.343ex;" alt="{\displaystyle \mathrm {F} _{1}=2\cdot {\frac {\mathrm {PPV} \cdot \mathrm {TPR} }{\mathrm {PPV} +\mathrm {TPR} }}={\frac {2\mathrm {TP} }{2\mathrm {TP} +\mathrm {FP} +\mathrm {FN} }}}"></span></dd>
<dt><a href="/wiki/Matthews_correlation_coefficient" title="Matthews correlation coefficient">Matthews correlation coefficient</a> (MCC)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {MCC} ={\frac {\mathrm {TP} \times \mathrm {TN} -\mathrm {FP} \times \mathrm {FN} }{\sqrt {(\mathrm {TP} +\mathrm {FP} )(\mathrm {TP} +\mathrm {FN} )(\mathrm {TN} +\mathrm {FP} )(\mathrm {TN} +\mathrm {FN} )}}}}">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">M</mi>
          <mi mathvariant="normal">C</mi>
          <mi mathvariant="normal">C</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mfrac>
            <mrow>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>×<!-- × --></mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>−<!-- − --></mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>×<!-- × --></mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
            </mrow>
            <msqrt>
              <mo stretchy="false">(</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo stretchy="false">)</mo>
              <mo stretchy="false">(</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo stretchy="false">)</mo>
              <mo stretchy="false">(</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">P</mi>
              </mrow>
              <mo stretchy="false">)</mo>
              <mo stretchy="false">(</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">T</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo>+</mo>
              <mrow class="MJX-TeXAtom-ORD">
                <mi mathvariant="normal">F</mi>
                <mi mathvariant="normal">N</mi>
              </mrow>
              <mo stretchy="false">)</mo>
            </msqrt>
          </mfrac>
        </mrow>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {MCC} ={\frac {\mathrm {TP} \times \mathrm {TN} -\mathrm {FP} \times \mathrm {FN} }{\sqrt {(\mathrm {TP} +\mathrm {FP} )(\mathrm {TP} +\mathrm {FN} )(\mathrm {TN} +\mathrm {FP} )(\mathrm {TN} +\mathrm {FN} )}}}}</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/6b556899b23f4222eca9ff369dac34f9ddc40a76" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -3.171ex; width:56.432ex; height:6.509ex;" alt="{\displaystyle \mathrm {MCC} ={\frac {\mathrm {TP} \times \mathrm {TN} -\mathrm {FP} \times \mathrm {FN} }{\sqrt {(\mathrm {TP} +\mathrm {FP} )(\mathrm {TP} +\mathrm {FN} )(\mathrm {TN} +\mathrm {FP} )(\mathrm {TN} +\mathrm {FN} )}}}}"></span></dd>
<dt><a href="/wiki/Informedness" class="mw-redirect" title="Informedness">Informedness</a> or Bookmaker Informedness (BM)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {BM} =\mathrm {TPR} +\mathrm {TNR} -1}">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">B</mi>
          <mi mathvariant="normal">M</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">T</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
        <mo>+</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">T</mi>
          <mi mathvariant="normal">N</mi>
          <mi mathvariant="normal">R</mi>
        </mrow>
        <mo>−<!-- − --></mo>
        <mn>1</mn>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {BM} =\mathrm {TPR} +\mathrm {TNR} -1}</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/d22a2922619a6c89d44d035af67633b8d17f24f3" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -0.505ex; width:23.822ex; height:2.343ex;" alt="{\displaystyle \mathrm {BM} =\mathrm {TPR} +\mathrm {TNR} -1}"></span></dd>
<dt>Markedness (MK)</dt>
<dd><span class="mwe-math-element"><span class="mwe-math-mathml-inline mwe-math-mathml-a11y" style="display: none;"><math xmlns="http://www.w3.org/1998/Math/MathML" alttext="{\displaystyle \mathrm {MK} =\mathrm {PPV} +\mathrm {NPV} -1}">
  <semantics>
    <mrow class="MJX-TeXAtom-ORD">
      <mstyle displaystyle="true" scriptlevel="0">
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">M</mi>
          <mi mathvariant="normal">K</mi>
        </mrow>
        <mo>=</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">V</mi>
        </mrow>
        <mo>+</mo>
        <mrow class="MJX-TeXAtom-ORD">
          <mi mathvariant="normal">N</mi>
          <mi mathvariant="normal">P</mi>
          <mi mathvariant="normal">V</mi>
        </mrow>
        <mo>−<!-- − --></mo>
        <mn>1</mn>
      </mstyle>
    </mrow>
    <annotation encoding="application/x-tex">{\displaystyle \mathrm {MK} =\mathrm {PPV} +\mathrm {NPV} -1}</annotation>
  </semantics>
</math></span><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/5437f65a40714949105525ad5f8217d654299bca" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -0.505ex; width:23.859ex; height:2.343ex;" alt="{\displaystyle \mathrm {MK} =\mathrm {PPV} +\mathrm {NPV} -1}"></span></dd></dl>
<p><span style="font-size:90%;"><i>Sources: Fawcett (2006),<sup id="cite_ref-Fawcett2006_3-0" class="reference"><a href="#cite_note-Fawcett2006-3">[3]</a></sup> Powers (2011),<sup id="cite_ref-Powers2011_4-0" class="reference"><a href="#cite_note-Powers2011-4">[4]</a></sup> Ting (2011),<sup id="cite_ref-ting2011precision_5-0" class="reference"><a href="#cite_note-ting2011precision-5">[5]</a></sup> and CAWCR<sup id="cite_ref-cawcr_6-0" class="reference"><a href="#cite_note-cawcr-6">[6]</a></sup></i>
</span>
</p>
</td></tr>

In [ ]:
single_sensors_states_dict, EF_states, LFA_states, LFL_states = get_states_arrays(standard_X_test)

for i, (train_fold_lst, test_fold_lst) in enumerate(zip(train_folds_list, test_folds_list)):
    gc.collect()
    
    # Preprocess the data
    train_fold_df, test_fold_df = get_folds_train_and_test(data, train_fold_lst, test_fold_lst)
    X_fold_train, X_fold_test, y_fold_train, y_fold_test = \
                split_fold_data_to_features_and_labels(train_fold_df, test_fold_df)
    standard_X_train, standard_X_test = standard_data_scaling(X_fold_train, X_fold_test)
    
    # Get rows with all sensors data
    y_fold_test.reset_index(drop=True, inplace=True)
    
    for _, sensor_cols_name_in_data in sensor_names.items():
        idx = standard_X_test[sensor_cols_name_in_data].isnull().all(1)
        idx_to_drop = standard_X_test[idx].index
        standard_X_test.drop(idx_to_drop, axis=0, inplace=True)
        y_fold_test.drop(idx_to_drop, axis=0, inplace=True)

#     standard_X_test, y_fold_test = get_rows_with_all_sensors_data(standard_X_test, y_fold_test)

    handle_nulls_in_X(standard_X_train, standard_X_test)
    
    single_sensor_models = classifiers['single_sensor_classifier'][i]
    
    # Eeach singlke sensor model
    get_single_sensor_state(single_sensors_states_dict, standard_X_test, y_fold_test, single_sensor_models)

    # LFA
    LFA_states = LFA_states + get_LFA_state(standard_X_test, y_fold_test,  single_sensor_models)
    
    # LFL
    LFL_states = LFL_states + get_LFL_state(standard_X_train, y_fold_train, standard_X_test, y_fold_test, single_sensor_models)

2019-10-18 15:45:31,579 - DEBUG - MainThread - Classifiers - <module> - First time initialize logger!


DEBUG:Classifiers:First time initialize logger!


35479

210

In [ ]:
evalutions_dict = dict()

# Single sensor
feature_names = get_feature_names(standard_X_test, ['label'])  # In this case we using the data with our label!
sensor_names = get_sensor_names(feature_names)

for sensor_name in sensor_names:
    single_sesnor_state = single_sensors_states_dict[sensor_name]
    sensitivity, specifisity, accuracy, precision, BA, F1 =\
        get_evaluations_metric_scores(single_sesnor_state[0],
                                      single_sesnor_state[1], single_sesnor_state[2], single_sesnor_state[3])
    insert_values_to_evaluations_dict(evalutions_dict, sensor_name, 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

# LFA
sensitivity, specifisity, accuracy, precision, BA, F1 =\
    get_evaluations_metric_scores(LFA_states[0], LFA_states[1], LFA_states[2], LFA_states[3])
insert_values_to_evaluations_dict(evalutions_dict, "LFA", 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

# LFL
sensitivity, specifisity, accuracy, precision, BA, F1 =\
    get_evaluations_metric_scores(LFL_states[0], LFL_states[1], LFL_states[2], LFL_states[3])
insert_values_to_evaluations_dict(evalutions_dict, "LFL", 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

pd.DataFrame.from_dict(evalutions_dict)